# Optimisation problem
* Maximise sth with a set of constraints
* Computational challenging, but can find a pretty good approximate solution
* 2 variances
    * 0-1
    * continuous or fractional (considerably easier)

# Problem statement:
* Each item is represented by a pair, <value, weight>
* The knapsack can accommodate items with a total weight of no more than w
* A vector, L, of length n, represents the set of available items. Each element of the vector is an item
* A vector, V, of length n, is used to indicate whether or not items are taken. If V[i] = 1, item I[i]is taken. If V[i] = 0, item I[i]is not taken
* Find a V that maximise $$\sum_{i=0}^{n-1} V[i]*I[i].value$$ subject to the constraint that $$\sum_{i=0}^{n-1} V[i]*I[i].weight <= w$$ 

In [1]:
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.calories
    def density(self):
        return self.getValue()/self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                 + ', ' + str(self.calories) + '>'

def buildMenu(names, values, calories):
    """names, values, calories lists of same length.
       name a list of strings
       values and calories lists of numbers
       returns list of Foods"""
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i],
                          calories[i]))
    return menu

names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)

# Bruteforce algorithm
1. Generate powerset
2. Remove all combinations whose total units exceeds the allowed weight
3. From the remaining, choose any one whose value is the greatest
O(2^n): note that 0/1 knapsack is inherently exponential

In [ ]:
def powerSet(items):
    N = len(items)
    # enumerate the 2**N possible combinations
    for i in range(2**N):
        combo = []
        for j in range(N):
            # test bit jth of integer i
            if (i >> j) % 2 == 1:
                combo.append(items[j])
        yield combo

# Greedy algorithm - a practical alternative
* metric: most valuable/lightest/largest density
* Pros
    * Easy to implement
    * computationally efficient: O(nlogn)
* Con: may not give the best answer - only give a local max

In [4]:
def greedy(items, maxCost, keyFunction):
    """Assumes items a list, maxCost >= 0,
         keyFunction maps elements of items to numbers"""
    itemsCopy = sorted(items, key = keyFunction,
                       reverse = True)  #O(nlogn) #sorted: return a copy
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):  #O(n)
        if (totalCost+itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('   ', item)

def testGreedys(foods, maxUnits):
    print('Use greedy by value to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits,
               lambda x: 1/Food.getCost(x))
    print('\nUse greedy by density to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.density)


names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)
testGreedys(foods, 750)

Use greedy by value to allocate 750 calories
Total value of items taken = 284.0
    burger: <100, 354>
    pizza: <95, 258>
    wine: <89, 123>

Use greedy by cost to allocate 750 calories
Total value of items taken = 318.0
    apple: <50, 95>
    wine: <89, 123>
    cola: <79, 150>
    beer: <90, 154>
    donut: <10, 195>

Use greedy by density to allocate 750 calories
Total value of items taken = 318.0
    wine: <89, 123>
    beer: <90, 154>
    cola: <79, 150>
    apple: <50, 95>
    donut: <10, 195>


# Search-Tree implementation
* Left-first, depth first enumeration
    * The first element is selected from the still to be considered items
        * If there is room for that item in the knapsack, a node is constructed that reflects the consequence of choosing to take that item. By convention, we draw that as the left child
        * We also explore the consequences of not taking that item. This is the right child
    * The process is then applied recursivelyto non-leaf children
    * Finally, chose a node with the highest value that meets constraints
* O(2^n), but an obvious optimization: don’t explore parts of tree that violate constraint

In [3]:
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake = maxVal(toConsider[1:],
                                     avail - nextItem.getCost())
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

def testMaxVal(foods, maxUnits, printItems = True):
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = maxVal(foods, maxUnits)
    print('Total value of items taken =', val)
    if printItems:
        for item in taken:
            print('   ', item)

testMaxVal(foods, 750)


Use search tree to allocate 750 calories
Total value of items taken = 353
    cola: <79, 150>
    pizza: <95, 258>
    beer: <90, 154>
    wine: <89, 123>


# Dynamic programming
* When does it work?
    * Optimal substructure: a globally optimal solution can be found by combining optimal solutions to local subproblems
    * Overlapping subproblems: finding an optimal solution involves solving the same problem multiple times
* although time complexity is still O(2^n), it often yields good performance for a subclass of optimization problems—those with optimal substructure and overlapping subproblems
    * Solution always correct
    * Fast under the right circumstances

In [3]:
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

for i in range(121):
    print('fib(' + str(i) + ') =', fastFib(i))

fib(0) = 1
fib(1) = 1
fib(2) = 2
fib(3) = 3
fib(4) = 5
fib(5) = 8
fib(6) = 13
fib(7) = 21
fib(8) = 34
fib(9) = 55
fib(10) = 89
fib(11) = 144
fib(12) = 233
fib(13) = 377
fib(14) = 610
fib(15) = 987
fib(16) = 1597
fib(17) = 2584
fib(18) = 4181
fib(19) = 6765
fib(20) = 10946
fib(21) = 17711
fib(22) = 28657
fib(23) = 46368
fib(24) = 75025
fib(25) = 121393
fib(26) = 196418
fib(27) = 317811
fib(28) = 514229
fib(29) = 832040
fib(30) = 1346269
fib(31) = 2178309
fib(32) = 3524578
fib(33) = 5702887
fib(34) = 9227465
fib(35) = 14930352
fib(36) = 24157817
fib(37) = 39088169
fib(38) = 63245986
fib(39) = 102334155
fib(40) = 165580141
fib(41) = 267914296
fib(42) = 433494437
fib(43) = 701408733
fib(44) = 1134903170
fib(45) = 1836311903
fib(46) = 2971215073
fib(47) = 4807526976
fib(48) = 7778742049
fib(49) = 12586269025
fib(50) = 20365011074
fib(51) = 32951280099
fib(52) = 53316291173
fib(53) = 86267571272
fib(54) = 139583862445
fib(55) = 225851433717
fib(56) = 365435296162
fib(57) = 591286729879
fib(5

In [11]:
import random
def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the 0/1 knapsack problem and the subjects of that solution
       Memo is a dict with key being a tuple(#items left to be considered, available weight)"""
    if (len(toConsider), avail) in memo:                #check whether it is already solved
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result              # update memo
    return result

def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print('Menu contains', len(foods), 'items')
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print('Total value of items taken =', val)
        for item in taken:
            print('   ', item)
            
def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxCost)))
    return items

for numItems in (2, 4, 8, 16, 32, 64, 128, 256, 512, 1024):
    numCalls = 0
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, fastMaxVal)

Menu contains 2 items
Use search tree to allocate 750 calories
Total value of items taken = 121
    1: <52, 71>
    0: <69, 197>
Menu contains 4 items
Use search tree to allocate 750 calories
Total value of items taken = 157
    3: <34, 161>
    2: <29, 249>
    1: <23, 43>
    0: <71, 112>
Menu contains 8 items
Use search tree to allocate 750 calories
Total value of items taken = 420
    7: <87, 119>
    6: <87, 107>
    5: <80, 247>
    4: <88, 63>
    1: <78, 167>
Menu contains 16 items
Use search tree to allocate 750 calories
Total value of items taken = 667
    7: <87, 119>
    6: <87, 107>
    4: <88, 63>
    10: <61, 94>
    6: <79, 84>
    5: <85, 36>
    2: <75, 44>
    1: <28, 141>
    0: <77, 18>
Menu contains 32 items
Use search tree to allocate 750 calories
Total value of items taken = 667
    7: <87, 119>
    6: <87, 107>
    4: <88, 63>
    10: <61, 94>
    6: <79, 84>
    5: <85, 36>
    2: <75, 44>
    1: <28, 141>
    0: <77, 18>
Menu contains 64 items
Use search tree

In [12]:
def countingFastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the subjects of that solution"""
    global numCalls
    numCalls += 1
    
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = countingFastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 countingFastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = countingFastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

for numItems in (2, 4, 8, 16, 32, 64, 128, 256, 512, 1024):
    numCalls = 0
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, countingFastMaxVal, False)
    print('Number of calls =', numCalls)

Menu contains 2 items
Use search tree to allocate 750 calories
Number of calls = 7
Menu contains 4 items
Use search tree to allocate 750 calories
Number of calls = 25
Menu contains 8 items
Use search tree to allocate 750 calories
Number of calls = 415
Menu contains 16 items
Use search tree to allocate 750 calories
Number of calls = 4237
Menu contains 32 items
Use search tree to allocate 750 calories
Number of calls = 23395
Menu contains 64 items
Use search tree to allocate 750 calories
Number of calls = 33242
Menu contains 128 items
Use search tree to allocate 750 calories
Number of calls = 91842
Menu contains 256 items
Use search tree to allocate 750 calories
Number of calls = 176123
Menu contains 512 items
Use search tree to allocate 750 calories
Number of calls = 355298
Menu contains 1024 items
Use search tree to allocate 750 calories
Number of calls = 697820
